# Single Class RetinaNet Auto-Annotator

In [ ]:
from scripts.inference import *
from scripts.preprocessing import *
from scripts.auto_annotation import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
#Check the contents of ../datasets. First ensure you place all the datasets to be annotated in ../datasets. 
get_all_datasets('./datasets')

In [ ]:
#Choose one of the datasets, e.g. SS_rhino
folder = 'demo/'
#specify class name
classes = {"rhinoceros"} 
#select your confidence threshold. A confidence threshold of 50 means all detections with confidence >=50 will be retained.
user_selected_confidence = 50

In [ ]:
#choose the auto-annotator. 
model_dir = './pretrained_models'
chosen_model = 'single_class_annotator.h5'
#load the model
model = choose_inference_model(model_dir, chosen_model)
class_mapping_file = os.path.join(model_dir,chosen_model[:-2]+"csv")
write_class_file(class_mapping_file, classes)
confidence_threshold = set_confidence_threshold(user_selected_confidence) 
unannotated_dataset = os.path.join('./datasets',folder)
images_to_annotate = generate_list_of_images(unannotated_dataset) 

In [ ]:
for each_im in images_to_annotate:
    #perform object detection 
    processed_image, object_detection_data = inference_per_image(unannotated_dataset, model, each_im, class_mapping_file, confidence_threshold)
    annotation = RetinaNet_Auto_Annotator(unannotated_dataset, each_im, object_detection_data, processed_image.shape, confidence_threshold)
    #write the xml file for this image
    annotation.write_xml_file()
  
    #displays image. comment out if you don't want visualisation.
    img = Image.fromarray(processed_image, 'RGB')
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(img)
    plt.show()